In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/MyDrive/Remi/

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1WJLnwO7mUn1QFbhFIJ6fMZjZWuCKf1ub/Remi


In [2]:
import pandas as pd
import numpy as np
import os 
import re
import operator
import pickle
import nltk 
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
import os.path
import json
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Data loading

In [55]:
i = 0
dfs = []
dir = 'wiki_annotation_pages/en/'
for filename in os.listdir(dir):
    data = pd.read_json(dir + filename)  
    data = data["query"]["pages"][0]
    data = pd.DataFrame([data])
    dfs.append(data)

temp = pd.concat(dfs, ignore_index=True)
temp.rename(columns={'title':'Subject'}, inplace=True)
temp.rename(columns={'extract':'content'}, inplace=True)


df_wikipedia =temp[['Subject','content']]
print(df_wikipedia)


                                              Subject  \
0                       COVID-19 vaccination in India   
1            Johns Hopkins Center for Health Security   
2                                         AstraZeneca   
3                            Janssen COVID-19 vaccine   
4                                    COVID-19 vaccine   
5                                      Michael Yeadon   
6       COVID-19 vaccine misinformation and hesitancy   
7                                       Nasal vaccine   
8                  COVID-19 vaccine clinical research   
9                              Variants of SARS-CoV-2   
10                                           COVID-19   
11                                         Hank Aaron   
12  Statistics of the COVID-19 pandemic in the Uni...   
13                 COVID-19 drug repurposing research   
14                                            Vaccine   
15                       COVID-19 pandemic in Florida   
16            List of lawsuits 

## Data Cleaning and Pre-processing

In [56]:
df_wikipedia['content']=[entry.lower() for entry in df_wikipedia['content']]
SW=['<span>', '<li>', '</li>', '<ul>', '</ul>', '<p>']
for sw in SW:
    df_wikipedia.content=df_wikipedia.content.replace(to_replace=sw,value='',regex=True)

df_wikipedia.content =df_wikipedia.content.replace(to_replace='<[^<]+?>',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='href"',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='-',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='\s+',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='\n',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='</p>',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='  ',value='',regex=True)
df_wikipedia.content =df_wikipedia.content.apply(lambda x:x.strip())

## data cleaning for subject data 
df_wikipedia.Subject =df_wikipedia.Subject.replace(to_replace='[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
df_wikipedia.Subject =df_wikipedia.Subject.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df_wikipedia.Subject =df_wikipedia.Subject.replace(to_replace='  ',value='',regex=True)    #remove double white space
df_wikipedia.Subject =df_wikipedia.Subject.apply(lambda x:x.strip())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


### Word Tokenization

In [57]:
df_wikipedia['Word tokenize']= [word_tokenize(entry) for entry in df_wikipedia.content]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Word Lemmatization

In [58]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
def wordLemmatizer(data):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    file_clean_k =pd.DataFrame()
    for index,entry in enumerate(data):
        
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if len(word)>1 and word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
            # The final processed set of words for each iteration will be stored in 'text_final'
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                #file_clean_k=file_clean_k.replace(to_replace ="\[.", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ="'", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace =" ", value = '', regex = True)
                #file_clean_k=file_clean_k.replace(to_replace ='\]', value = '', regex = True)
    return file_clean_k

In [59]:
df_wikipedia.shape 

(36, 3)

In [60]:
df_clean = wordLemmatizer(df_wikipedia['Word tokenize'][0:36]) 
df_clean

,Keyword_final
0,"['india', 'begin', 'administration', 'covid', ..."
1,"['john', 'hopkins', 'center', 'health', 'secur..."
2,"['astrazeneca', 'plc', 'british', 'swedish', '..."
3,"['janssen', 'vaccine', 'sell', 'brand', 'name'..."
4,"['vaccine', 'vaccine', 'intend', 'provide', 'a..."
5,"['michael', 'yeadon', 'british', 'anti', 'vacc..."
6,"['anti', 'vaccination', 'activist', 'people', ..."
7,"['nasal', 'vaccine', 'vaccine', 'administer', ..."
8,"['covid', 'vaccine', 'clinical', 'research', '..."
9,"['many', 'variant', 'severe', 'acute', 'respir..."


In [61]:
df_clean=df_clean.replace(to_replace ="\[.", value = '', regex = True)
df_clean=df_clean.replace(to_replace ="'", value = '', regex = True)
df_clean=df_clean.replace(to_replace =" ", value = '', regex = True)
df_clean=df_clean.replace(to_replace ='\]', value = '', regex = True)

### Added WordLemmatize words into given dataframe

In [62]:
## Insert New column in df_wikipedia to stored the Clean Keyword
df_wikipedia.insert(loc=3, column='Clean_Keyword', value=df_clean['Keyword_final'].tolist())

In [63]:
df_wikipedia_save= df_wikipedia
df_wikipedia_save = df_wikipedia_save.drop(['Word tokenize','Clean_Keyword'],axis=1)
df_wikipedia_save

,Subject,content
0,COVID 19 vaccination in India,india began administration of covid 19 vaccine...
1,Johns Hopkins Center for Health Security,the johns hopkins center for health security a...
2,AstraZeneca,astrazeneca plc is a british swedish multinati...
3,Janssen COVID 19 vaccine,the janssen covid‑19 vaccine sold under the br...
4,COVID 19 vaccine,a covid‑19 vaccine is a vaccine intended to pr...
5,Michael Yeadon,michael yeadon is a british anti vaccine activ...
6,COVID 19 vaccine misinformation and hesitancy,anti vaccination activists and other people in...
7,Nasal vaccine,a nasal vaccine is a vaccine administered to a...
8,COVID 19 vaccine clinical research,covid 19 vaccine clinical research uses clinic...
9,Variants of SARS CoV 2,there are many variants of severe acute respir...


In [14]:
#df_wikipedia_save.to_csv("df_wikipedia_index.csv", index=False, header=True)

In [64]:
## Sample of clean keywords
df_wikipedia.Clean_Keyword[0]

'india,begin,administration,covid,vaccine,january,june,india,administer,billion,dos,overall,include,first,second,precautionary,booster,dos,currently,approve,vaccine,india,eligible,population,receive,least,one,shot,eligible,population,fully,vaccinate,india,initially,approve,vaccine,manufacture,license,serum,institute,india,trade,name,covishield,covaxin,vaccine,develop,locally,bharat,biotech,since,join,sputnik,manufacture,license,reddy,laboratory,additional,production,serum,institute,india,start,september,moderna,vaccine,johnson,amp,johnson,vaccine,zycov,vaccine,locally,develop,zydus,cadila,vaccine,candidate,undergo,local,clinical,trial,accord,june,study,publish,lancet,covid,vaccination,india,prevent,additional,million,death,december,december,vaccination,programme,cumulative,dos,administer,across,country,total,dos,administer,across,country,july,monthly,graph,cumulative,dos,administer,across,country,graph,daily,dos,administer,across,country,graph,daily,dos,administer,last,updated,july,vac

# 1. Create document search engine with TF-IDF

### TF-IDF by using  TfidfVectorizer from sklearn.feature_extraction.text

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator

## Create Vocabulary
vocabulary = set()

for doc in df_wikipedia.Clean_Keyword:
    vocabulary.update(doc.split(','))

vocabulary = list(vocabulary)

# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=vocabulary,dtype=np.float32)

print(tfidf)

# Fit the TfIdf model
tfidf.fit(df_wikipedia.Clean_Keyword)

# Transform the TfIdf model
tfidf_tran=tfidf.transform(df_wikipedia.Clean_Keyword)


TfidfVectorizer(dtype=<class 'numpy.float32'>,
                vocabulary=['counselor', 'condition', 'demonstrator', 'genus',
                            'organism', 'consumption', 'anaphylaxis', 'panel',
                            'supervision', 'kitasato', 'delight', 'essential',
                            'conspire', 'prioritize', 'scene', 'nicotiana',
                            'contraindication', 'ethnicity', 'sentencing',
                            'regional', 'john', 'tangible', 'question',
                            'detectable', 'ribbon', 'dimyristoyl',
                            'transparently', 'staple', 'doxycycline',
                            'apparently', ...])


### Save Trained TFIDF Model

In [66]:
### Save model
with open('tfid.pkl','wb') as handle:
    pickle.dump(tfidf_tran, handle)

In [67]:
### load model
t = pickle.load(open('tfid.pkl','rb'))

### Save Vocabulary


In [68]:
### Save Vacabulary
with open("vocab_wiki_files.txt", "w") as file:
    file.write(str(vocabulary))

In [69]:
### load Vacabulary
with open("vocab_wiki_files.txt", "r") as file:
    data2 = eval(file.readline())

### Create vector for Query/search keywords

In [70]:
## Create vector for Query/search keywords
def gen_vector_T(tokens):
    Q = np.zeros((len(vocabulary)))
    x= tfidf.transform(tokens)
    for token in tokens[0].split(','):
        try:
            ind = vocabulary.index(token)
            Q[ind]  = x[0, tfidf.vocabulary_[token]]
        except:
            pass
    return Q

### Calculate Cosine Similarity with formula 

In [71]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

### Calculate Cosine similarity of trained Tfidf to input query

In [72]:
def cosine_similarity_T(k, query):
    #print("Cosine Similarity")
    preprocessed_query = preprocessed_query = re.sub("\W+", " ", query).strip()
    tokens = word_tokenize(str(preprocessed_query))
    q_df = pd.DataFrame(columns=['q_clean'])
    q_df.loc[0,'q_clean'] =tokens
    q_df['q_clean'] =wordLemmatizer(q_df.q_clean)
    q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
    q_df=q_df.replace(to_replace ="'", value = '', regex = True)
    q_df=q_df.replace(to_replace =" ", value = '', regex = True)
    q_df=q_df.replace(to_replace ='\]', value = '', regex = True)

    d_cosines = []

    query_vector = gen_vector_T(q_df['q_clean'])
    
    for d in tfidf_tran.A:
        
        d_cosines.append(cosine_sim(query_vector, d))
                    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    #print("")
    d_cosines.sort()
    #print(out)
    a = pd.DataFrame()
    for i,index in enumerate(out):
        a.loc[i,'index'] = str(index)
        a.loc[i,'Subject'] = df_wikipedia['Subject'][index]
    for j,simScore in enumerate(d_cosines[-k:][::-1]):
        a.loc[j,'Score'] = simScore
    return a

In [73]:
%time cosine_similarity_T(10,'Bihar')
#df_wikipedia['Subject']

CPU times: user 27.7 ms, sys: 10.2 ms, total: 38 ms
Wall time: 24.9 ms


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


,index,Subject,Score
0,35,List of COVID 19 vaccine authorizations,NaN
1,34,Human rights in Qatar,NaN
2,15,COVID 19 pandemic in Florida,NaN
3,14,Vaccine,NaN
4,13,COVID 19 drug repurposing research,NaN
5,12,Statistics of the COVID 19 pandemic in the Uni...,NaN
6,11,Hank Aaron,NaN
7,10,COVID 19,NaN
8,9,Variants of SARS CoV 2,NaN
9,8,COVID 19 vaccine clinical research,NaN


In [22]:
%time cosine_similarity_T(10,'Together with collaborators, we founded a spinout company to develop and commercialise a nasal spray vaccine against COVID. The vaccine is based on research by our and @UniEastFinland researchers. #HelsinkiHealth #corona #COVID19 #covidvaccine')

CPU times: user 45.1 ms, sys: 41 ms, total: 86.1 ms
Wall time: 49 ms


,index,Subject,Score
0,21,COVID 19 vaccine,0.381987
1,27,CureVac,0.181993
2,15,COVID 19 pandemic in the United States,0.069331
3,2,COVID 19 misinformation,0.057964
4,3,COVID 19,0.057501
5,5,COVID 19 misinformation by China,0.052688
6,4,Chinese government response to COVID 19,0.044074
7,23,List of unproven methods against COVID 19,0.042369
8,12,COVID 19 testing,0.034105
9,7,COVID 19 pandemic in Belgium,0.024901


#Using Google

In [82]:
!pip install --upgrade tensorflow-gpu
#Install TF-Hub.
!pip install tensorflow-hub
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [81]:
#!pip3 install tensorflow-cpu --upgrade --force-reinstall
#!pip install -q tensorflow_text==2.4.1
!pip3 install tensorflow_text
import tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:
import pandas as pd
import numpy as np
import re, string
import os 
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import linear_kernel

module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
model = hub.load(module_url)

#print(model.max_seq_length)
# # #Create function for using modeltraining
def embed(input):
    return model(input)


print(df_wikipedia['content'][1])

Model_USE= embed(df_wikipedia['content'])

exported = tf.train.Checkpoint(v=tf.Variable(Model_USE))
exported.f = tf.function(
    lambda  x: exported.v * x,
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])

tf.saved_model.save(exported,'TrainModel')

imported = tf.saved_model.load('TrainModel')
loadedmodel =imported.v.numpy()
loadedmodel.shape

def SearchDocument(query):
    q =[query]
    # embed the query for calcluating the similarity
    Q_Train =embed(q)

    # Calculate the Similarity
    linear_similarities = linear_kernel(Q_Train, loadedmodel).flatten() 
    #Sort top 10 index with similarity score
    Top_index_doc = linear_similarities.argsort()[:-11:-1]
    # sort by similarity score
    linear_similarities.sort()
    a = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        a.loc[i,'index'] = str(index)
        a.loc[i,'File_Name'] = df_wikipedia['Subject'][index] ## Read File name with index from File_data DF
    for j,simScore in enumerate(linear_similarities[:-11:-1]):
        a.loc[j,'Score'] = simScore
    return a

SearchDocument('Nasal vaccine')

the johns hopkins center for health security abbreviated chs is an independent nonprofit organization of the johns hopkins bloomberg school of public health. the center works to protect people’s health from epidemics and pandemics and ensures that communities are resilient to major challenges. the center is also concerned with biological weapons and the biosecurity implications of emerging biotechnology. the center for health security gives policy recommendations to the united states government the world health organization and the un biological weapons convention. history the center for health security began as the johns hopkins center for civilian biodefense strategies ccbs in 1998 at the johns hopkins bloomberg school of public health. d. a. henderson served as the founding director. at that time the center was the first and only academic center focused on biosecurity policy and practice. at one point around 2003 chs had become part of a new umbrella organization called the institut

,index,File_Name,Score
0,7,Nasal vaccine,0.458511
1,33,Oxford–AstraZeneca COVID 19 vaccine,0.385976
2,6,COVID 19 vaccine misinformation and hesitancy,0.378889
3,0,COVID 19 vaccination in India,0.373719
4,4,COVID 19 vaccine,0.373092
5,14,Vaccine,0.367734
6,8,COVID 19 vaccine clinical research,0.356319
7,25,Deployment of COVID 19 vaccines,0.346365
8,35,List of COVID 19 vaccine authorizations,0.345054
9,3,Janssen COVID 19 vaccine,0.343193


In [104]:
import re
import scipy
import heapq
import statistics

# Load covidcheck
covid_check = pd.read_csv('noheadings_en_merged_clef_emnlp.csv', sep=',')
index = 0
limiter = 0
matches = 0
cosine_list = []

i = 0
dfs = []
dir = 'wiki_annotation_pages/en/'
for filename in os.listdir(dir):
    data = pd.read_json(dir + filename)  
    data = data["query"]["pages"][0]
    filename = filename.replace('.json', '')
    data.update({'filename': filename})
    data = pd.DataFrame([data])
    dfs.append(data)

temp = pd.concat(dfs, ignore_index=True)
temp.rename(columns={'title':'Subject'}, inplace=True)
temp.rename(columns={'extract':'content'}, inplace=True)

df_wikipedia =temp[['Subject','content', 'filename']]
#print(df_wikipedia)

df_wikipedia['content']=[entry.lower() for entry in df_wikipedia['content']]
SW=['<span>', '<li>', '</li>', '<ul>', '</ul>', '<p>']
for sw in SW:
    df_wikipedia.content=df_wikipedia.content.replace(to_replace=sw,value='',regex=True)

df_wikipedia.content =df_wikipedia.content.replace(to_replace='<[^<]+?>',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='href"',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='-',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='\s+',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='\n',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='</p>',value=' ',regex=True)
df_wikipedia.content =df_wikipedia.content.replace(to_replace='  ',value='',regex=True)
df_wikipedia.content =df_wikipedia.content.apply(lambda x:x.strip())

## data cleaning for subject data 
df_wikipedia.Subject =df_wikipedia.Subject.replace(to_replace='[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]',value=' ',regex=True)
df_wikipedia.Subject =df_wikipedia.Subject.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df_wikipedia.Subject =df_wikipedia.Subject.replace(to_replace='  ',value='',regex=True)    #remove double white space
df_wikipedia.Subject =df_wikipedia.Subject.apply(lambda x:x.strip())

def similarity_huggingface(query):
  # Calculatte similarity for huggingface models
  Q_Train = model.encode(q)
  linear_similarities = scipy.spatial.distance.cdist(Q_Train, embeddings, "cosine")[0]
  linear_similarities = 1 - linear_similarities
  return linear_similarities

def similarity_tfhub(query):
  # Calculatte similarity for Tensorflow Hub models
  Q_Train =embed(q)
  #linear_similarities = linear_kernel(Q_Train, loadedmodel).flatten()
  linear_similarities = scipy.spatial.distance.cdist(Q_Train, loadedmodel, "cosine")[0]
  linear_similarities = 1 - linear_similarities
  return linear_similarities

def tfidf_similarity():
  index = 0
  matches = 0
  for checkworthy in covid_check['class_label']:
    correct_docs = []
    if checkworthy == 1:
      # input check-worthy message
      query = covid_check['tweet_text'][index]
      preprocessed_query = re.sub("\W+", " ", query).strip()
      tokens = word_tokenize(str(preprocessed_query))

      q_df = pd.DataFrame(columns=['q_clean'])
      q_df.loc[0,'q_clean'] =tokens
      q_df['q_clean'] =wordLemmatizer(q_df.q_clean)
      q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
      q_df=q_df.replace(to_replace ="'", value = '', regex = True)
      q_df=q_df.replace(to_replace =" ", value = '', regex = True)
      q_df=q_df.replace(to_replace ='\]', value = '', regex = True)
      
      # check if the correct document exists
      if type(covid_check['verify_url'][index]) == str:
        correct_docs = re.findall(r"wiki/([^/\n\n ]+)", covid_check['verify_url'][index])
      
      d_cosines = []
      query_vector = gen_vector_T(q_df['q_clean'])
      for d in tfidf_tran.A:
        d_cosines.append(cosine_sim(query_vector, d))
      # top 10 matches
      Top_index_doc = np.array(d_cosines).argsort()[-1:][::-1]
      #print(len(Top_index_doc))
      # Check if retrieved top 10 contains correct document
      #print("top docs: ", Top_index_doc)
      predicted_docs = [df_wikipedia['filename'][index] for index in Top_index_doc]
      #print(predicted_docs)
      np.array(d_cosines)
      #print(np.array(d_cosines))
      if max(np.array(d_cosines)) < 0.14:
        predicted_docs = [None for i in predicted_docs]
        #print(predicted_docs)
      # Check if the correct answer is in the prediction

      flat_cosines = np.array(d_cosines).flatten()
      flat_cosines.sort()
      flat_cosines = flat_cosines[-10:]
      cosine_list.extend(flat_cosines)

      if len(correct_docs) > 0:
        for doc in correct_docs:
          if doc in predicted_docs:
            matches += 1
      else:
        if None in predicted_docs:
          for i in predicted_docs:
            matches += 1
    index += 1
  print("Amount correct: ", matches)

# tfidf_similarity()

for checkworthy in covid_check['class_label']:
  correct_docs = []
  if checkworthy == 1:
    # input check-worthy message
    q =[covid_check['tweet_text'][index]]
    
    # check if the correct document exists
    if type(covid_check['verify_url'][index]) == str:
      correct_docs = re.findall(r"wiki/([^/\n\n ]+)", covid_check['verify_url'][index])
      #print(correct_docs)
    
    linear_similarities = similarity_huggingface(q)
    # top 10 matches
    Top_index_doc = linear_similarities.argsort()[:-11:-1]
    # Check if retrieved top 10 contains correct document
    #print("top docs: ", Top_index_doc)
    predicted_docs = [df_wikipedia['filename'][index] for index in Top_index_doc]
    #print(predicted_docs)
    
    #print(linear_similarities)
    linear_similarities.sort()
    linear_similarities = linear_similarities[-10:]
    cosine_list.extend(linear_similarities)
    #print(heapq.nlargest(1, linear_similarities))
    if max(linear_similarities) < 0.39:
        predicted_docs = [None for i in predicted_docs]
        print(predicted_docs)

    # Check if the correct answer is in the prediction
    if len(correct_docs) > 0:
      for doc in correct_docs:
        if doc in predicted_docs:
          print(len(correct_docs))
          matches += 1
    else:
      if None in predicted_docs:
        for i in predicted_docs:
          print("hoi!")
          matches += 1
  index += 1

print(statistics.mean(cosine_list))
print("Amount correct: ", matches)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1
2
1
2
1
1
2
2
1
1
3
1
2
2
2
[None, None, None, None, None, None, None, None, None, None]
1
1
1
1
[None, None, None, None, None, None, None, None, None, None]
1
2
2
1
1
1
[None, None, None, None, None, None, None, None, None, None]
1
4
[None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None]
hoi!
hoi!
hoi!
hoi!
hoi!
hoi!
hoi!
hoi!
hoi!
hoi!
[None, None, None, None, None, None, None, None, None, None]
1
[None, None, None, None, None, None, None, None, None, None]
hoi!
hoi!
hoi!
hoi!
hoi!
hoi!
hoi!
hoi!
hoi!
hoi!
0.38763684387185243
Amount correct:  48


In [96]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [97]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import linear_kernel
from numpy import dot
from numpy.linalg import norm
import scipy

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
#model = SentenceTransformer('sentence-transformers/allenai-specter')

model.max_seq_length = 512
print("Max Sequence Length:", model.max_seq_length)
embeddings = model.encode(df_wikipedia['content'])

def SearchDocument(query):
    q =[query]
    # embed the query for calcluating the similarity
    Q_Train = model.encode(q)

    # Calculate the Similarity
    #linear_similarities = linear_kernel(Q_Train, embeddings).flatten()
    linear_similarities = scipy.spatial.distance.cdist(Q_Train, embeddings, "cosine")[0]
    linear_similarities = 1 - linear_similarities
    #Sort top 10 index with similarity score
    Top_index_doc = linear_similarities.argsort()[:-11:-1]
    # sort by similarity score
    linear_similarities.sort()
    a = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        a.loc[i,'index'] = str(index)
        a.loc[i,'File_Name'] = df_wikipedia['Subject'][index] ## Read File name with index from File_data DF
    for j,simScore in enumerate(linear_similarities[:-11:-1]):
        a.loc[j,'Score'] = simScore
    return a

#SearchDocument('johnson and johnson vaccines derived from murdered babies. at least 2 other vaccines are not. if you take vaccines use those.')

#SearchDocument('Michael Yeadon	is a former person that was important to someone')

#SearchDocument('Together with collaborators, we founded a spinout company to develop and commercialise a nasal spray vaccine against COVID. The vaccine is based on research by our and @UniEastFinland researchers. #HelsinkiHealth #corona #COVID19 #covidvaccine')

Max Sequence Length: 512


## Longformer

In [ ]:
import scipy
from transformers import LongformerTokenizer, LongformerModel
from sklearn.metrics.pairwise import linear_kernel
import torch

from sklearn.metrics.pairwise import cosine_similarity
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerModel.from_pretrained("allenai/longformer-base-4096")

model.max_seq_length = 100

def cls_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    model_out = token_embeddings.mean(axis=1)
    return model_out

def cls_pooling_too(model_output, attention_mask):
    return model_output.last_hidden_state[:,0]

wikidata = df_wikipedia['content'].tolist() 

encoded_input = tokenizer(wikidata, return_tensors="pt", max_length=100, truncation=True, padding=True)
#output = model(**encoded_input)

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = cls_pooling(model_output, encoded_input['attention_mask'])

def SearchDocument(query):
    q =[query]
    # embed the query for calcluating the similarity
    Q_Train = tokenizer(q, return_tensors="pt", max_length=100, truncation=True, padding=True)
    with torch.no_grad():
      model_output = model(**Q_Train)
    
    query_embedding = cls_pooling(model_output, encoded_input['attention_mask'])

    # Calculate the Similarity
    #linear_similarities = linear_kernel(query_embedding, sentence_embeddings).flatten()
    # linear_similarities = scipy.spatial.distance.cdist(query_embedding, sentence_embeddings, "cosine")[0]
    # linear_similarities = 1 - linear_similarities
    linear_similarities = torch.cosine_similarity(query_embedding, sentence_embeddings)
    
    #Sort top 10 index with similarity score
    Top_index_doc = linear_similarities.argsort()[:-11:-1]
    # sort by similarity score
    linear_similarities.sort()
    a = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        a.loc[i,'index'] = str(index)
        a.loc[i,'File_Name'] = df_wikipedia['Subject'][index] ## Read File name with index from File_data DF
    for j,simScore in enumerate(linear_similarities[:-11:-1]):
        a.loc[j,'Score'] = simScore
    return a

SearchDocument('Ivermectin is a thing')

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ValueError: ignored

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 27.7 MB/s 
     |████████████████████████████████| 6.6 MB 56.2 MB/s 
     |████████████████████████████████| 120 kB 72.0 MB/s 


In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
